In [223]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
from functools import partial
import re

from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt
import numpy as np

In [224]:
partitions = 12
parlog = sc.textFile("/home/milroy/pyspark/lustre_debug.out", partitions)

In [225]:
parlog.take(10)

[u'00010000:00080000:2.1F:1433384402.983324:0:0:0:(ldlm_lib.c:2008:target_recovery_expired()) scratch-MDT0000: recovery timed out; 2 clients are still in recovery after 300s (136 clients connected)',
 u'00000100:00080000:22.0:1433439189.202419:0:28364:0:(service.c:789:ptlrpc_update_export_timer()) updating export e0b948f3-2c66-de79-82f6-858c54bcf73f at 1433439189 exp ffff8805feae1000',
 u'00000100:00080000:22.0:1433439189.205134:0:28364:0:(service.c:789:ptlrpc_update_export_timer()) updating export 1915dbbb-3d37-10c8-c161-d533b76bcbcb at 1433439189 exp ffff8805cf6dec00',
 u'00000100:00080000:22.0:1433439189.208987:0:28364:0:(service.c:789:ptlrpc_update_export_timer()) updating export 37147439-d85e-5002-0501-0315cbd3a063 at 1433439189 exp ffff8805cf5f7800',
 u'00000100:00080000:22.0:1433439189.213463:0:28364:0:(service.c:789:ptlrpc_update_export_timer()) updating export ed7c79f9-b42b-ce30-ba52-38106df6aaed at 1433439189 exp ffff8805b6a0dc00',
 u'00000100:00080000:22.0:1433439189.215960:

In [326]:
words = parlog.filter(lambda line: line.count('-') > 3).filter(
    lambda line: 'updating' in line).map(lambda line: re.split('\W+', line.lower().strip()))

In [343]:
words.take(3)

[[u'00000100',
  u'00080000',
  u'22',
  u'0',
  u'1433439189',
  u'202419',
  u'0',
  u'28364',
  u'0',
  u'service',
  u'c',
  u'789',
  u'ptlrpc_update_export_timer',
  u'updating',
  u'export',
  u'e0b948f3',
  u'2c66',
  u'de79',
  u'82f6',
  u'858c54bcf73f',
  u'at',
  u'1433439189',
  u'exp',
  u'ffff8805feae1000'],
 [u'00000100',
  u'00080000',
  u'22',
  u'0',
  u'1433439189',
  u'205134',
  u'0',
  u'28364',
  u'0',
  u'service',
  u'c',
  u'789',
  u'ptlrpc_update_export_timer',
  u'updating',
  u'export',
  u'1915dbbb',
  u'3d37',
  u'10c8',
  u'c161',
  u'd533b76bcbcb',
  u'at',
  u'1433439189',
  u'exp',
  u'ffff8805cf6dec00'],
 [u'00000100',
  u'00080000',
  u'22',
  u'0',
  u'1433439189',
  u'208987',
  u'0',
  u'28364',
  u'0',
  u'service',
  u'c',
  u'789',
  u'ptlrpc_update_export_timer',
  u'updating',
  u'export',
  u'37147439',
  u'd85e',
  u'5002',
  u'0501',
  u'0315cbd3a063',
  u'at',
  u'1433439189',
  u'exp',
  u'ffff8805cf5f7800']]

In [329]:
exports = words.flatMap(lambda line: [[int(line[7]), str(x), str(''.join(line[15:19]))] 
                                      for x in line if x.startswith('ffff')])

In [330]:
exports.take(1)

[[28364, 'ffff8805feae1000', 'e0b948f32c66de7982f6']]

In [331]:
to_int = exports.map(lambda row: [row[0], int(row[1], 16), int(row[2], 16)])

In [332]:
to_int.take(10)

[[28364, 18446612158061875200L, 1061227996410447197209334L],
 [28364, 18446612157269142528L, 118462377013510696124769L],
 [28364, 18446612157268195328L, 260107466402186330768641L],
 [28364, 18446612156853050368L, 1121497041959493926173266L],
 [28364, 18446612144951080960L, 251071831362941308278518L],
 [28364, 18446612144976318464L, 735877783481886252866594L],
 [28364, 18446612156822798336L, 213260712583334543733914L],
 [28364, 18446612156934072320L, 538490657162060706488576L],
 [28364, 18446612157129886720L, 471359184725092152589577L],
 [28364, 18446612156607994880L, 734911401062701910371150L]]

In [333]:
to_vector = to_int.map(lambda row: np.asarray([float(x) for x in row]))

In [342]:
to_vector.take(2)

[array([  2.83640000e+04,   1.84466122e+19,   1.06122800e+24]),
 array([  2.83640000e+04,   1.84466122e+19,   1.18462377e+23])]

In [344]:
from pyspark.mllib.stat import Statistics

In [347]:
pearsonCorr = Statistics.corr(to_vector, method="spearman")

In [348]:
pearsonCorr

array([[  1.00000000e+00,  -2.93146845e-03,  -2.03278004e-04],
       [ -2.93146845e-03,   1.00000000e+00,  -2.72207813e-02],
       [ -2.03278004e-04,  -2.72207813e-02,   1.00000000e+00]])

In [234]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import LabeledPoint 

In [243]:
to_vector.union(to_vector).take(1)

[array([  2.83640000e+04,   1.84466122e+19])]

In [339]:
labpt = to_vector.map(lambda arr: LabeledPoint(arr[0], arr[1:]))

In [340]:
labpt.take(1)

[LabeledPoint(28364.0, [1.84466121581e+19,1.06122799641e+24])]

In [341]:
model = LogisticRegressionWithSGD.train(labpt)

Py4JJavaError: An error occurred while calling o1810.trainLogisticRegressionModelWithSGD.
: org.apache.spark.SparkException: Input validation failed.
	at org.apache.spark.mllib.regression.GeneralizedLinearAlgorithm.run(GeneralizedLinearAlgorithm.scala:225)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:81)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainLogisticRegressionModelWithSGD(PythonMLLibAPI.scala:240)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand